<a href="https://colab.research.google.com/github/neerapari-code/dog-vs-cat-classification/blob/main/CIFAR100_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [4]:
model = models.resnet18(pretrained=True)
model.fc = nn.Identity()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 190MB/s]


model = models.inception_v3(pretrained=True)
model.fc = nn.Identity()

In [5]:
for param in model.parameters():
    param.requires_grad = False

In [6]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [7]:
image_datasets = {x: datasets.CIFAR100(root='./data', train=(x=='train'), download=True,
                                       transform=data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64, shuffle=False, num_workers=0) for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

100%|██████████| 169M/169M [00:03<00:00, 48.9MB/s]


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [9]:
def accu(model, dataloader):
    model = model.eval()
    acc = 0
    total = 0
    for data, target in dataloader:
        o = model(data.to(device))
        total += target.to(device).size(0)
        acc += (o.argmax(dim=1).long() == target.to(device)).sum().item()
    return acc / total

In [12]:
accu(model,dataloaders_dict['train'])

2e-05

In [10]:
accu(model,dataloaders_dict['val'])

0.0

In [13]:
import pandas as pd
features = []
tags = []
#model = model.to(device)
with torch.no_grad():
    for images, labels in dataloaders_dict['train']:
        outputs = model(images.to(device))
        features.append(outputs.cpu().data.numpy())
        tags.append(labels.cpu().numpy())

In [14]:
train_feature = pd.concat([pd.DataFrame(x) for x in features],axis=0,ignore_index=True)
train_label = np.concatenate(tags)
train_label = pd.Series(train_label)

In [16]:
import os
import pandas as pd

# ... (your existing code) ...

# Create the 'processed_data' directory if it doesn't exist
os.makedirs('./processed_data', exist_ok=True)

train_feature.to_csv('./processed_data/cifar100_train_resnet18.csv',index=False,header=False)
train_label.to_csv('./processed_data/cifar100_train_label_resnet18.csv',index=False,header=False)

In [17]:
features_test = []
tags_test = []
#model = model.to(device)
with torch.no_grad():
    for data, target in dataloaders_dict['val']:
        outputs = model(data.to(device))
        features_test.append(outputs.cpu().data.numpy())
        tags_test.append(target.cpu().numpy())

In [18]:
test_feature = pd.concat([pd.DataFrame(x) for x in features_test],axis=0,ignore_index=True)
test_label = np.concatenate(tags_test)
test_label = pd.Series(test_label)

In [19]:
test_feature.to_csv('./processed_data/cifar100_test_resnet18.csv',index=False,header=False)
test_label.to_csv('./processed_data/cifar100_test_label_resnet18.csv',index=False,header=False)